- rimozione outlier
- rimozione errori (-1 su delta) --> NaN
- controllare che non ci siano righe con valori completmente nulli
- rimozione cobbled, gravel perchè sono tutti true e false
- rimuovere i ciclisti dal dataframe dei ciclisti che non hanno partecipato a gare ???
- rimozione delle feature correlate 
- drop temperatura

# Data Cleaning